# Data Filtering for Non-time-series modelling
Important events file exists as a midpoint, so that the filtering can be done in chunks, necessary for low RAM.

In [33]:
importantEventsFile = 'data/importantChartEvents.csv'
sourceEventsFile = 'data/chartevents.csv'
finalEventsFile = 'data/chartEventsPred.csv'
itemIDsFile = 'importantItemIDs.txt'
itemLabelsFile = 'data/d_items.csv'

## Reading in important features
Read in item ids of important features

In [10]:
import pandas as pd

with open(itemIDsFile, 'r') as f:
    importantItemIDs = f.read().splitlines()

importantItemIDs = [int(x) for x in importantItemIDs]
print(importantItemIDs)

[220045, 220048, 220050, 220051, 220052, 220179, 220180, 220181, 220210, 220227, 220228, 220274, 220277, 220341, 220344, 220363, 220364, 220395, 220451, 220454, 220467, 220546, 220603, 220615, 220621, 220645, 220650, 223762, 224167, 224643, 225170, 225624, 225664, 225667, 225693, 225695, 225948, 226537, 226996, 227242, 227243, 227463, 227466, 228699, 229761, 230093]


Filter events file for only these features

In [11]:
with open(importantEventsFile, 'w') as f:
    f.write(
        'subject_id,charttime,itemid,value,valuenum\n'
    )

for chunk in pd.read_csv(sourceEventsFile, usecols=['subject_id', 'charttime', 'itemid', 'valuenum'], chunksize=100000):
    chunk = chunk[chunk['itemid'].isin(importantItemIDs)]
    chunk.to_csv(importantEventsFile, mode='a', header=False, index=False)

In [12]:
chartEvents = pd.read_csv(importantEventsFile)
chartEvents.head()

,subject_id,charttime,itemid,value,valuenum
0,10000032,2180-07-23 14:00:00,220048,SR (Sinus Rhythm),NaN
1,10000032,2180-07-23 14:11:00,220179,84,84.0
2,10000032,2180-07-23 14:11:00,220180,48,48.0
3,10000032,2180-07-23 14:11:00,220181,56,56.0
4,10000032,2180-07-23 14:12:00,220045,91,91.0


Keep only numerical data

In [13]:
chartEvents.rename(columns={'valuenum': 'value'}, inplace=True)
chartEvents['value'] = chartEvents['value'].to_numeric().dropna()
chartEvents['value'] = chartEvents['value'].astype(float)

chartEvents.head()

,subject_id,charttime,itemid,value
0,10000032,2180-07-23 14:00:00,220048,SR (Sinus Rhythm)
1,10000032,2180-07-23 14:11:00,220179,84.0
2,10000032,2180-07-23 14:11:00,220180,48.0
3,10000032,2180-07-23 14:11:00,220181,56.0
4,10000032,2180-07-23 14:12:00,220045,91.0


In [14]:
chartEvents.isna().sum()

subject_id    0
charttime     0
itemid        0
value         0
dtype: int64

In [15]:
print(chartEvents.shape)

(65414228, 4)


Add feature labels

In [16]:
d_items = pd.read_csv(itemLabelsFile, usecols=['itemid', 'label'])

chartEvents = chartEvents.merge(d_items, on='itemid')
chartEvents.head()

,subject_id,charttime,itemid,value,label
0,10000032,2180-07-23 14:00:00,220048,SR (Sinus Rhythm),Heart Rhythm
1,10000032,2180-07-23 14:30:00,220048,SR (Sinus Rhythm),Heart Rhythm
2,10000032,2180-07-23 15:00:00,220048,SR (Sinus Rhythm),Heart Rhythm
3,10000032,2180-07-23 16:00:00,220048,ST (Sinus Tachycardia),Heart Rhythm
4,10000032,2180-07-23 17:00:00,220048,SR (Sinus Rhythm),Heart Rhythm


In [17]:
chartEvents = chartEvents[['subject_id', 'label', 'charttime', 'value']]
chartEvents = chartEvents.sort_values(['subject_id', 'label', 'charttime'])
chartEvents.to_csv(importantEventsFile, index=False)

In [18]:
chartEvents.head()

,subject_id,label,charttime,value
51835242,10000032,BUN,2180-07-23 21:45:00,33.0
50032379,10000032,Creatinine (serum),2180-07-23 21:45:00,0.5
50623245,10000032,Glucose (serum),2180-07-23 21:45:00,115.0
24076640,10000032,Heart Rate,2180-07-23 14:12:00,91.0
24076641,10000032,Heart Rate,2180-07-23 14:30:00,93.0


Checkpoint if previously ran the above

In [2]:
import pandas as pd
chartEvents = pd.read_csv('data/importantChartEvents.csv')

In [3]:
chartEvents.head()

,subject_id,charttime,itemid,value
0,10000032,2180-07-23 14:00:00,223761,98.7
1,10000032,2180-07-23 14:11:00,220179,84.0
2,10000032,2180-07-23 14:11:00,220180,48.0
3,10000032,2180-07-23 14:11:00,220181,56.0
4,10000032,2180-07-23 14:12:00,220045,91.0


## Blood Glucose Time-series capture
Separate glucose values for time series processing

In [8]:
glucose_df = chartEvents[chartEvents['label'] == 'Glucose']
glucose_df = glucose_df.sort_values(by=['subject_id', 'charttime'])
glucose_df.head()

,subject_id,charttime,value,label
44730996,10000032,2180-07-23 21:45:00,115.0,Glucose
44730997,10000690,2150-11-03 02:56:00,77.0,Glucose
44730998,10000690,2150-11-04 03:03:00,84.0,Glucose
44730999,10000690,2150-11-04 17:54:00,120.0,Glucose
44731000,10000690,2150-11-05 05:36:00,107.0,Glucose


In [9]:
glucose_df.describe()

,subject_id,value
count,5.848410e+05,584841.000000
mean,1.500637e+07,263.823095
std,2.893124e+06,10938.316108
min,1.000003e+07,0.000000
25%,1.251347e+07,106.000000
50%,1.501423e+07,129.000000
75%,1.751638e+07,162.000000
max,1.999999e+07,999999.000000


In [10]:
glucose_df = glucose_df[glucose_df['value'] <= 1000]
glucose_df = glucose_df[glucose_df['value'] >= 20]

for each glucose reading, find the proceeding reading as the prediction value

In [11]:
# Shift value to get the "next" reading
glucose_df['next_glucose'] = glucose_df.groupby('subject_id')['value'].shift(-1)
glucose_df.head()

,subject_id,charttime,value,label,next_glucose
44730996,10000032,2180-07-23 21:45:00,115.0,Glucose,NaN
44730997,10000690,2150-11-03 02:56:00,77.0,Glucose,84.0
44730998,10000690,2150-11-04 03:03:00,84.0,Glucose,120.0
44730999,10000690,2150-11-04 17:54:00,120.0,Glucose,107.0
44731000,10000690,2150-11-05 05:36:00,107.0,Glucose,97.0


In [12]:
print(glucose_df.isnull().sum())
print(glucose_df.shape)

subject_id          0
charttime           0
value               0
label               0
next_glucose    63738
dtype: int64
(584399, 5)


Assign it a classification label based on final glucose

In [13]:
# Classify next glucose value
def classify_glucose(val):
    if pd.isna(val):
        return None
    if val < 70:
        return 'hypo'
    elif val > 180:
        return 'hyper'
    else:
        return 'normal'

glucose_df['label'] = glucose_df['next_glucose'].apply(classify_glucose)
glucose_df.head()

,subject_id,charttime,value,label,next_glucose
44730996,10000032,2180-07-23 21:45:00,115.0,None,NaN
44730997,10000690,2150-11-03 02:56:00,77.0,normal,84.0
44730998,10000690,2150-11-04 03:03:00,84.0,normal,120.0
44730999,10000690,2150-11-04 17:54:00,120.0,normal,107.0
44731000,10000690,2150-11-05 05:36:00,107.0,normal,97.0


Derive statistic characteristics from the previous 3 glucose (before the current and prediction)

In [14]:
# Example: calculate rolling features per subject
glucose_df['mean_last3'] = glucose_df.groupby('subject_id')['value'].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)
glucose_df['std_last3'] = glucose_df.groupby('subject_id')['value'].rolling(3, min_periods=1).std().reset_index(level=0, drop=True)
glucose_df['trend'] = glucose_df.groupby('subject_id')['value'].diff()  # slope-ish
glucose_df.head()

,subject_id,charttime,value,label,next_glucose,mean_last3,std_last3,trend
44730996,10000032,2180-07-23 21:45:00,115.0,None,NaN,115.000000,NaN,NaN
44730997,10000690,2150-11-03 02:56:00,77.0,normal,84.0,77.000000,NaN,NaN
44730998,10000690,2150-11-04 03:03:00,84.0,normal,120.0,80.500000,4.949747,7.0
44730999,10000690,2150-11-04 17:54:00,120.0,normal,107.0,93.666667,23.072350,36.0
44731000,10000690,2150-11-05 05:36:00,107.0,normal,97.0,103.666667,18.230012,-13.0


In [15]:
glucose_df = glucose_df.dropna(subset=['label'])
glucose_df.head()

,subject_id,charttime,value,label,next_glucose,mean_last3,std_last3,trend
44730997,10000690,2150-11-03 02:56:00,77.0,normal,84.0,77.000000,NaN,NaN
44730998,10000690,2150-11-04 03:03:00,84.0,normal,120.0,80.500000,4.949747,7.0
44730999,10000690,2150-11-04 17:54:00,120.0,normal,107.0,93.666667,23.072350,36.0
44731000,10000690,2150-11-05 05:36:00,107.0,normal,97.0,103.666667,18.230012,-13.0
44731002,10001217,2157-11-21 03:16:00,113.0,normal,113.0,113.000000,NaN,NaN


In [17]:
glucose_df.dropna(inplace=True)
glucose_df.isnull().sum()

subject_id      0
charttime       0
value           0
label           0
next_glucose    0
mean_last3      0
std_last3       0
trend           0
dtype: int64

In [18]:
glucose_df.to_csv('data/glucose.csv', index=False)

## Patient vitals
Average out and pivot the rest of the patient's vital information

In [19]:
pivot_df = chartEvents.pivot_table(
    index='subject_id',
    columns='label',
    values='value',
    aggfunc='mean'
)

pivot_df.head()

label,Anion gap,BP Diastolic,BP Mean,BP Systolic,BUN,Creatinine (serum),Glucose,Heart Rate,Hematocrit (serum),Hemoglobin,Magnesium,O2 saturation pulseoxymetry,Respiratory Rate,Sodium (serum),Temperature Fahrenheit,WBC
subject_id,,,,,,,,,,,,,,,,
10000032,14.0,54.100000,62.300000,88.900000,33.0,0.50,115.0,96.500000,NaN,NaN,2.300000,96.300000,20.700000,132.0,98.966667,NaN
10000690,11.6,60.361702,74.936170,122.893617,20.2,0.82,97.0,84.072917,30.475,10.175,1.883333,95.702128,22.557895,135.2,97.736364,6.35
10000980,NaN,83.272727,97.545455,142.454545,NaN,NaN,NaN,73.636364,NaN,NaN,NaN,98.909091,20.545455,NaN,98.133333,NaN
10001217,14.0,77.720000,88.673469,126.900000,9.5,0.45,113.0,86.711538,35.550,11.750,1.850000,95.019231,19.038462,138.5,98.693333,13.70
10001725,15.0,61.187500,70.062500,100.406250,17.0,0.80,149.0,79.156250,37.700,13.250,2.150000,98.225806,17.531250,139.0,97.900000,18.55


In [20]:
print(pivot_df.shape)
print(pivot_df.isnull().sum())

(65366, 16)
label
Anion gap                      1569
BP Diastolic                    664
BP Mean                         692
BP Systolic                     663
BUN                            1543
Creatinine (serum)             1533
Glucose                        1625
Heart Rate                        1
Hematocrit (serum)             1623
Hemoglobin                     1713
Magnesium                      2534
O2 saturation pulseoxymetry      62
Respiratory Rate                 64
Sodium (serum)                 1519
Temperature Fahrenheit          925
WBC                            1734
dtype: int64


Dropped the averaged glucose (to be replaced with more in depth readings)

In [21]:
pivot_df.drop(columns=['Glucose'], inplace=True)
pivot_df.head()

label,Anion gap,BP Diastolic,BP Mean,BP Systolic,BUN,Creatinine (serum),Heart Rate,Hematocrit (serum),Hemoglobin,Magnesium,O2 saturation pulseoxymetry,Respiratory Rate,Sodium (serum),Temperature Fahrenheit,WBC
subject_id,,,,,,,,,,,,,,,
10000032,14.0,54.100000,62.300000,88.900000,33.0,0.50,96.500000,NaN,NaN,2.300000,96.300000,20.700000,132.0,98.966667,NaN
10000690,11.6,60.361702,74.936170,122.893617,20.2,0.82,84.072917,30.475,10.175,1.883333,95.702128,22.557895,135.2,97.736364,6.35
10000980,NaN,83.272727,97.545455,142.454545,NaN,NaN,73.636364,NaN,NaN,NaN,98.909091,20.545455,NaN,98.133333,NaN
10001217,14.0,77.720000,88.673469,126.900000,9.5,0.45,86.711538,35.550,11.750,1.850000,95.019231,19.038462,138.5,98.693333,13.70
10001725,15.0,61.187500,70.062500,100.406250,17.0,0.80,79.156250,37.700,13.250,2.150000,98.225806,17.531250,139.0,97.900000,18.55


Remove outliers from the patient vitals

In [22]:
def remove_outliers_iqr(df, multiplier=1.5):
    numeric_cols = df.select_dtypes(include='number').columns
    mask = pd.Series(True, index=df.index)

    for col in numeric_cols:
        q1 = col.quantile(0.25)
        q3 = col.quantile(0.75)
        iqr = q3 - q1
        # Keep rows where all numeric columns are within IQR range
        mask &= (df[col] >= (q1 - multiplier * iqr)) & (df[col] <= (q3 + multiplier * iqr))

    return df[mask]

pivot_df = remove_outliers_iqr(pivot_df)

Add in the earlier found glucose readings along with patient vital information

In [23]:
merged_df = pd.merge(glucose_df, pivot_df, on='subject_id')
merged_df.head()

,subject_id,charttime,value,label,next_glucose,mean_last3,std_last3,trend,Anion gap,BP Diastolic,...,Creatinine (serum),Heart Rate,Hematocrit (serum),Hemoglobin,Magnesium,O2 saturation pulseoxymetry,Respiratory Rate,Sodium (serum),Temperature Fahrenheit,WBC
0,10000690,2150-11-04 03:03:00,84.0,normal,120.0,80.500000,4.949747,7.0,11.600000,60.361702,...,0.820000,84.072917,30.475000,10.175000,1.883333,95.702128,22.557895,135.2,97.736364,6.35
1,10000690,2150-11-04 17:54:00,120.0,normal,107.0,93.666667,23.072350,36.0,11.600000,60.361702,...,0.820000,84.072917,30.475000,10.175000,1.883333,95.702128,22.557895,135.2,97.736364,6.35
2,10000690,2150-11-05 05:36:00,107.0,normal,97.0,103.666667,18.230012,-13.0,11.600000,60.361702,...,0.820000,84.072917,30.475000,10.175000,1.883333,95.702128,22.557895,135.2,97.736364,6.35
3,10001884,2131-01-12 03:34:00,145.0,hyper,199.0,146.500000,2.121320,-3.0,10.214286,74.669683,...,0.885714,70.800000,27.226667,8.670588,2.228571,97.648649,19.044444,136.5,98.989130,12.78
4,10001884,2131-01-13 04:29:00,199.0,normal,177.0,164.000000,30.347982,54.0,10.214286,74.669683,...,0.885714,70.800000,27.226667,8.670588,2.228571,97.648649,19.044444,136.5,98.989130,12.78


In [26]:
for col in merged_df.columns:
    if merged_df[col].isnull().sum() > 0.05*merged_df[col].shape[0]:
        merged_df.drop([col], axis=1, inplace=True)

merged_df.dropna(inplace=True)
merged_df.rename(columns={'value': 'glucose'}, inplace=True)
print(merged_df.shape)

(271513, 23)


## Saving data files
File of all patient glucose readings

In [27]:
patients = merged_df[['subject_id', 'charttime', 'label', 'next_glucose']]
patients.head()

,subject_id,charttime,label,next_glucose
0,10000690,2150-11-04 03:03:00,normal,120.0
1,10000690,2150-11-04 17:54:00,normal,107.0
2,10000690,2150-11-05 05:36:00,normal,97.0
3,10001884,2131-01-12 03:34:00,hyper,199.0
4,10001884,2131-01-13 04:29:00,normal,177.0


In [28]:
patients.to_csv('data/patients.csv', index=False)

File for only dysglycemic patients

In [31]:
dys_patients = merged_df[(merged_df['label'] == 'hypo') | (merged_df['label'] == 'hyper')]
dys_patients.head()

,subject_id,charttime,glucose,label,next_glucose,mean_last3,std_last3,trend,Anion gap,BP Diastolic,...,Creatinine (serum),Heart Rate,Hematocrit (serum),Hemoglobin,Magnesium,O2 saturation pulseoxymetry,Respiratory Rate,Sodium (serum),Temperature Fahrenheit,WBC
3,10001884,2131-01-12 03:34:00,145.0,hyper,199.0,146.500000,2.121320,-3.0,10.214286,74.669683,...,0.885714,70.800000,27.226667,8.670588,2.228571,97.648649,19.044444,136.500000,98.989130,12.78
11,10001884,2131-01-16 04:02:00,135.0,hyper,203.0,136.000000,33.511192,-35.0,10.214286,74.669683,...,0.885714,70.800000,27.226667,8.670588,2.228571,97.648649,19.044444,136.500000,98.989130,12.78
19,10002114,2162-02-19 13:28:00,78.0,hypo,66.0,84.000000,7.211103,-4.0,10.428571,82.169231,...,1.785714,80.850000,30.737500,10.637500,2.271429,98.210526,16.746835,130.285714,97.557143,6.50
30,10002155,2131-03-09 23:02:00,166.0,hyper,235.0,132.333333,29.871949,44.0,13.285714,52.501873,...,1.507143,89.754266,28.588235,9.816667,2.000000,93.701342,17.276451,134.857143,97.308571,6.30
31,10002155,2131-03-10 02:04:00,235.0,hyper,192.0,174.333333,56.959050,69.0,13.285714,52.501873,...,1.507143,89.754266,28.588235,9.816667,2.000000,93.701342,17.276451,134.857143,97.308571,6.30


In [30]:
dys_patients.to_csv('data/dys_patients.csv', index=False)

Final deidentified dataset

In [34]:
merged_df.to_csv(finalEventsFile, index=False)
merged_df.head()

,subject_id,charttime,glucose,label,next_glucose,mean_last3,std_last3,trend,Anion gap,BP Diastolic,...,Creatinine (serum),Heart Rate,Hematocrit (serum),Hemoglobin,Magnesium,O2 saturation pulseoxymetry,Respiratory Rate,Sodium (serum),Temperature Fahrenheit,WBC
0,10000690,2150-11-04 03:03:00,84.0,normal,120.0,80.500000,4.949747,7.0,11.600000,60.361702,...,0.820000,84.072917,30.475000,10.175000,1.883333,95.702128,22.557895,135.2,97.736364,6.35
1,10000690,2150-11-04 17:54:00,120.0,normal,107.0,93.666667,23.072350,36.0,11.600000,60.361702,...,0.820000,84.072917,30.475000,10.175000,1.883333,95.702128,22.557895,135.2,97.736364,6.35
2,10000690,2150-11-05 05:36:00,107.0,normal,97.0,103.666667,18.230012,-13.0,11.600000,60.361702,...,0.820000,84.072917,30.475000,10.175000,1.883333,95.702128,22.557895,135.2,97.736364,6.35
3,10001884,2131-01-12 03:34:00,145.0,hyper,199.0,146.500000,2.121320,-3.0,10.214286,74.669683,...,0.885714,70.800000,27.226667,8.670588,2.228571,97.648649,19.044444,136.5,98.989130,12.78
4,10001884,2131-01-13 04:29:00,199.0,normal,177.0,164.000000,30.347982,54.0,10.214286,74.669683,...,0.885714,70.800000,27.226667,8.670588,2.228571,97.648649,19.044444,136.5,98.989130,12.78
